# Lab 6: Extracting Job Posts from Indeed

In [1]:
import pandas
import configparser
import psycopg2

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

host = config['myaws']['host']
database = config['myaws']['database']
user = config['myaws']['user']
pwd = config['myaws']['pwd']

In [3]:
conn = psycopg2.connect(host = host,
                       user = user,
                        password = pwd,
                        dbname = database
                       )
cur = conn.cursor()

In [4]:
table_sql = """
            CREATE TABLE IF NOT EXISTS indeed_gp9
            (
                id SERIAL,
                job_title VARCHAR(200),
                job_company VARCHAR(200),
                job_loc VARCHAR(200),
                job_salary VARCHAR(200),
                job_summary TEXT,
                PRIMARY KEY(id)
            );

            """

In [5]:
cur.execute(table_sql)
conn.commit()

## Request HTML

In [6]:
url = 'https://www.indeed.com/jobs?q=intelligence+analyst&start=40'

In [7]:
import urllib.request
response = urllib.request.urlopen(url)
html_data= response.read()
#print(html_data.decode('utf-8'))

## Parse HTML

In [8]:
!pip install beautifulsoup4

In [16]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_data,'html.parser')
#print (soup)

In [10]:
for table_resultsBody in soup.find_all('table', id = 'resultsBody'):
    pass
    #print(table_resultsBody)

In [11]:
for table_pageContent in table_resultsBody.find_all('table', id = 'pageContent'):
    pass
    #print(table_pageContent)

In [12]:
for td_resultsCol in table_pageContent.find_all('td', id = 'resultsCol'):
    pass
    #print(td_resultsCol)

In [13]:
#  identify the job title, company, ratings, reviews, salary, and summary
for div_row in td_resultsCol.find_all('div', class_='jobsearch-SerpJobCard unifiedRow row result'):

    # find job title
    job_title = None
    job_company  = None
    job_rating = None
    job_loc = None
    job_salary = None
    job_summary = None
    for div_title in div_row.find_all('div', class_ = 'title'):
        job_title = div_title.text.strip().replace("'","_")
    
    for div_dsc in div_row.find_all('div', class_ = 'sjcl'):
        
        #find company name
        for span_company in div_dsc.find_all('span', class_ = 'company'):
            job_company = span_company.text.strip().replace("'","_")
        
        # find location
        for div_loc in div_dsc.find_all('div', class_ = 'location accessible-contrast-color-location'):
            job_loc = div_loc.text.strip().replace("'","_")
    
    # find salary
    for div_salary in div_row.find_all('div',class_ ='salarySnippet'):
        job_salary = div_salary.text.strip().replace("'","_")
    
    #find summary
    for div_summary in div_row.find_all('div', class_ = 'summary'):
        job_summary = div_summary.text.strip().replace("'","_")
        
    # insert into database
    
    sql_insert = """
                insert into indeed_gp9(job_title,job_company,job_loc,job_salary,job_summary)
                values('{}','{}','{}','{}','{}')            
                """.format(job_title,job_company,job_loc,job_salary,job_summary)

    cur.execute(sql_insert)
    conn.commit()

## View the Table

In [14]:
df = pandas.read_sql_query('select * from indeed_gp9', conn)
df[:]

,id,job_title,job_company,job_loc,job_salary,job_summary
0,1,Intelligence Analyst,National Security Agency,"Fort Meade, MD","$69,581 - $108,422 a year",NSA Intelligence Analysts are expected to take...
1,2,Undergraduate Internship/Co-op Program - Intel...,Central Intelligence Agency,"Washington, DC",$21.21 - $22.43 an hour,"Engineering, science students, or those in oth..."
2,3,H-ISAC Cyber Threat Intelligence Analyst,Perch Security,"Titusville, FL","$50,000 - $75,000 a year",Ability to communicate technical issues to tec...
3,4,Acoustic Intelligence Analyst,Perspecta,"Suitland, MD 20746",None,We provide solutions to improve the Nation_s d...
4,5,"Cybersecurity Analyst (DLP) (60-70k, US Citize...",Maveris,"Martinsburg, WV","$60,000 - $70,000 a year",The Analyst can identify risks in security sys...
5,6,Transportation Analyst,Lipman Family Farms,"Immokalee, FL 34142",None,"Perform fact based, data driven analysis, and ..."
6,7,Intelligence Analyst,Booz Allen Hamilton,None,None,We’re an EOE that empowers our people—no matte...
7,8,Intelligence Analyst I - MRS,WorldAware,None,None,WorldAware is dedicated to equal opportunity i...
8,9,Intelligence Analyst,BAE Systems,None,None,Having an understanding of physical and techni...
9,10,Intelligence Analyst,5 Stones intelligence,None,None,Analyze threat information from multiple sourc...


In [15]:
cur.close()
conn.close()